In [1]:
from interpret import show
from interpret.blackbox import ShapKernel
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
import sys
sys.path.append("..")
from evaluate import IoU_value
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from interpret import show
from interpret.blackbox import LimeTabular
import pandas as pd
import numpy as np
import xgboost

In [2]:
def normalize(x):
    if x.max() == x.min():
        return x
    else:
        result = (x-x.min())/(x.max()-x.min())
        return result

# 在生成的数据集中验证SHAP

In [2]:
df = pd.read_csv("train/train_data_0.csv")  #10000x100数据，不包含Target
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature90,feature91,feature92,feature93,feature94,feature95,feature96,feature97,feature98,feature99
0,84.071517,15,95,37,109.222187,41.054423,14,12.635977,14.010126,70,...,26,56,32,30,30,65,36.400093,55,44,11.299354
1,64.376450,12,39,40,108.267721,58.627029,46,80.431019,3.598875,70,...,28,38,69,28,10,44,32.313545,30,45,10.109159
2,61.747596,36,58,7,51.315514,10.865075,53,43.284407,52.822478,70,...,20,45,62,43,29,48,35.294801,27,70,11.926330
3,108.096804,32,73,59,40.416383,12.580249,70,19.908695,59.408556,70,...,26,7,57,68,3,42,37.391035,47,71,10.819648
4,93.659688,14,45,30,85.312636,47.390543,81,30.780931,44.901262,71,...,30,10,46,42,3,42,32.743209,42,47,11.301096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,46.443361,29,59,22,41.987071,57.629751,39,78.792761,4.834577,71,...,26,63,38,66,16,62,36.587602,31,50,7.942675
9996,101.995631,23,57,16,100.037233,41.170912,25,32.059358,31.253103,71,...,29,57,44,52,7,62,37.806850,31,70,11.128170
9997,79.100888,42,36,44,100.913728,59.806790,17,69.167360,47.454068,70,...,22,60,46,4,32,62,32.413842,52,59,10.260244
9998,97.844526,31,41,46,52.108113,51.004928,65,39.811859,36.990671,70,...,25,20,42,4,19,47,36.201993,27,54,8.643205


# 线性模型解释

$ y = 5 \times x_0 + 1 \times x_1 + 2 \times x_3$ 

In [5]:
df["label"] = df.feature0 *1 + df.feature1 * 1 + df.feature3*1
df["label"] = (normalize(df["label"])>0.5).astype(int)
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature91,feature92,feature93,feature94,feature95,feature96,feature97,feature98,feature99,label
0,84.071517,15,95,37,109.222187,41.054423,14,12.635977,14.010126,70,...,56,32,30,30,65,36.400093,55,44,11.299354,0
1,64.376450,12,39,40,108.267721,58.627029,46,80.431019,3.598875,70,...,38,69,28,10,44,32.313545,30,45,10.109159,0
2,61.747596,36,58,7,51.315514,10.865075,53,43.284407,52.822478,70,...,45,62,43,29,48,35.294801,27,70,11.926330,0
3,108.096804,32,73,59,40.416383,12.580249,70,19.908695,59.408556,70,...,7,57,68,3,42,37.391035,47,71,10.819648,1
4,93.659688,14,45,30,85.312636,47.390543,81,30.780931,44.901262,71,...,10,46,42,3,42,32.743209,42,47,11.301096,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,46.443361,29,59,22,41.987071,57.629751,39,78.792761,4.834577,71,...,63,38,66,16,62,36.587602,31,50,7.942675,0
9996,101.995631,23,57,16,100.037233,41.170912,25,32.059358,31.253103,71,...,57,44,52,7,62,37.806850,31,70,11.128170,1
9997,79.100888,42,36,44,100.913728,59.806790,17,69.167360,47.454068,70,...,60,46,4,32,62,32.413842,52,59,10.260244,1
9998,97.844526,31,41,46,52.108113,51.004928,65,39.811859,36.990671,70,...,20,42,4,19,47,36.201993,27,54,8.643205,1


In [6]:
data = df.iloc[:,:-1]
label = df.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(data,label,test_size = 0.3)

In [ ]:
xgb_clf = xgboost.XGBClassifier()
xgb_clf.fit(X_train,y_train)
shap = ShapKernel(predict_fn=xgb_clf.predict_proba, data=X_train.iloc[:3000],n_jobs=5)  # 多进程用多了内存容易爆
shap_local = shap.explain_local(X_test[:5], y_test[:5])
show(shap_local)

[13:47:43] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Using 3000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


## Lending Club 数据集

In [3]:
df = pd.read_csv("../train/true_dataset/lending club/lending_club_processed.csv",index_col=0)
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,15000.0,15000.0,15000.0,60.0,5.32,451.73,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,0.0,0.0,0.0
1,4800.0,4800.0,4800.0,60.0,7.99,150.40,0.0,4.0,10.0,2.0,...,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,0.0,0.0,1.0
2,15000.0,15000.0,15000.0,60.0,10.49,487.47,1.0,11.0,10.0,0.0,...,0.0,0.0,0.0,100922.0,17428.0,6300.0,17371.0,0.0,0.0,0.0
3,3000.0,3000.0,3000.0,60.0,5.32,90.35,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,254797.0,23666.0,61300.0,56497.0,0.0,0.0,0.0
4,8000.0,8000.0,8000.0,60.0,13.49,271.45,2.0,21.0,4.0,0.0,...,14.3,0.0,0.0,163222.0,25361.0,52400.0,7462.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57089,36400.0,36400.0,36400.0,60.0,14.49,856.24,2.0,23.0,5.0,1.0,...,75.0,0.0,0.0,529965.0,137277.0,43500.0,90665.0,0.0,0.0,1.0
57090,23800.0,23800.0,23800.0,60.0,14.49,559.85,2.0,23.0,10.0,0.0,...,100.0,0.0,0.0,406737.0,192571.0,61600.0,95737.0,0.0,0.0,0.0
57091,18000.0,18000.0,18000.0,60.0,9.49,377.95,1.0,11.0,5.0,2.0,...,33.3,0.0,0.0,275356.0,54349.0,13100.0,77756.0,0.0,0.0,0.0
57092,29400.0,29400.0,29400.0,60.0,13.99,683.94,2.0,22.0,9.0,0.0,...,42.9,0.0,0.0,719056.0,148305.0,56500.0,95702.0,0.0,0.0,0.0


In [12]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,:-1],df.iloc[:,-1],test_size=0.3,random_state=42)

In [13]:
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state=42)
X_res, y_res = smo.fit_resample(X_train, y_train)

In [14]:
xgb_clf = xgboost.XGBClassifier()
xgb_clf.fit(X_res,y_res)
#xgb_clf.fit(X_train,y_train)
xgb_clf.score(X_test,y_test)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[16:43:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9991826726604005

In [9]:
import datetime  #5000 x 95输入占用18gb内存 10000行占用32gb内存
for i in [1000,2000,5000,8000,10000]:
    start = datetime.datetime.now()
    shap = ShapKernel(predict_fn=xgb_clf.predict_proba, data=X_train.iloc[:i],n_jobs=15)  # 多进程用多了内存容易爆
    shap_local = shap.explain_local(X_test[:5], y_test[:5])
    show(shap_local)
    end = datetime.datetime.now()
    print('totally time for '+str(i)+ " is "+ str(end - start))
    

Using 1000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/5 [00:00<?, ?it/s]

Using 2000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


totally time for 1000 is 0:00:38.319331


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Using 10000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


totally time for 8000 is 0:05:35.138826


  0%|          | 0/5 [00:00<?, ?it/s]

MemoryError: Unable to allocate 15.6 GiB for an array with shape (2234, 940000) and data type float64

In [9]:
from transform import init_transform
init_transform(df,"meaning.txt")

name:  申请贷款金额
loan_amnt
name:  实际贷款金额
funded_amnt
name:  期限（单位月）
term
name:  贷款利率
int_rate
name:  贷款月支付额
installment
name:  信用评级
grade
name:  信用子评级
sub_grade
name:  工作年限
emp_length
name:  住房性质
home_ownership
name:  年收入
annual_inc
name:  收入来源是否核实
verification_status
name:  贷款目的
purpose
name:  地理位置
addr_state
name:  债务收入比
dti
name:  近两年逾期30天以上的次数
delinq_2yrs
name:  近6个月查询次数
inq_last_6mths
name:  自上一次逾期的月数
mths_since_last_delinq
name:  信用借款额度
open_acc
name:  公共事业记录差评的次数
pub_rec
name:  未偿还本金总额
out_prncp
name:  已偿还总额
total_pymnt
name:  已偿还本金
total_rec_prncp
name:  已偿还利息
total_rec_int
name:  催回总额
tot_coll_amt


In [15]:
shap = ShapKernel(predict_fn=xgb_clf.predict_proba, data=X_train.iloc[:5000],n_jobs=15)  # 多进程用多了内存容易爆
shap_local = shap.explain_local(X_test[:5], y_test[:5])
#show(shap_local)

Using 5000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
show(shap_local)